# Entrenamiento del modelo

### **1. Carga del dataset procesado**
Se utiliza el archivo limpio **`spotify_clean_modeling.csv`**, previamente generado en la etapa de preprocesamiento.

### **2. Separación de variables predictoras (X) y objetivo (y)**
- **X:** atributos musicales numéricos y categóricos.  
- **y:** variable binaria **`is_hit`**, que indica si una canción es considerada un “hit”.

### **3. Preparación del preprocesamiento**
- Identificación de columnas numéricas y categóricas.
- Aplicación de **OneHotEncoder** a las variables categóricas.
- Construcción de un **ColumnTransformer** para combinar:
  - Codificación One-Hot  
  - Paso directo de variables numéricas

### **4. Construcción del pipeline del modelo**
Se integra el preprocesador y el algoritmo dentro de un único **Pipeline**, garantizando un flujo reproducible y listo para producción.

### **5. División del conjunto de datos**
- Separación en entrenamiento y prueba mediante `train_test_split`.  
- Uso de `stratify=y` para mantener la proporción de clases.

### **6. Entrenamiento del modelo LightGBM**
- Ajuste del modelo utilizando `class_weight="balanced"` para corregir el fuerte desbalance de clases.  
- Entrenamiento del pipeline completo sobre el conjunto de entrenamiento.

### **7. Evaluación inicial del modelo**
Métricas calculadas sobre el conjunto de prueba:
- **Accuracy**  
- **F1-score** (clave por el desbalance)  
- **ROC-AUC**

### **8. Optimización del umbral de decisión (threshold tuning)**
- Evaluación del **F1-score** para múltiples umbrales.  
- Selección del threshold que maximiza la detección correcta de canciones exitosas.

### **9. Evaluación final con threshold optimizado**
- Comparación de métricas con el nuevo umbral seleccionado.  
- Confirmación de mejoras en la detección de la clase positiva (**hits**).

### **10. Guardado del modelo entrenado**
- Serialización del pipeline completo mediante **`joblib`**.  
- Generación del archivo **`lightgbm_hit_classifier.joblib`** para su uso futuro en:
  - Módulo de inferencia  
  - API `/songs/predict_hit`

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

import os
import joblib
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn import set_config


In [45]:

# Ruta al archivo fuente inicial 
DATA_PATH = "../data/processed/spotify_clean_modeling.csv"

# Verificar existencia
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró el archivo en {DATA_PATH}")

# Carga el archivo CSV
df = pd.read_csv(DATA_PATH)
print(f"Dataset se ha cargado correctamente en un arreglo: {df.shape}")
display(df.columns.T)
display(df.head())


Dataset se ha cargado correctamente en un arreglo: (232724, 20)


Index(['genre', 'popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'is_hit', 'duration_min', 'beat_density', 'energy_valence',
       'dance_energy', 'speech_valence', 'acoustic_energy', 'inst_energy',
       'dance_valence'],
      dtype='object')

,genre,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,is_hit,duration_min,beat_density,energy_valence,dance_energy,speech_valence,acoustic_energy,inst_energy,dance_valence
0,Movie,0,0.611,0.389,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814,0,1.656217,100.813501,0.740740,0.353990,0.042735,0.556010,0.000000,0.316646
1,Movie,1,0.246,0.590,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816,0,2.289550,75.998777,0.601392,0.434830,0.070829,0.181302,0.000000,0.481440
2,Movie,3,0.952,0.663,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368,0,2.837783,35.058350,0.048208,0.086853,0.013322,0.124712,0.000000,0.243984
3,Movie,0,0.703,0.240,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227,0,2.540450,67.609282,0.074002,0.078240,0.008967,0.229178,0.000000,0.054480
4,Movie,4,0.950,0.331,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390,0,1.377083,102.082421,0.087750,0.074475,0.017784,0.213750,0.027675,0.129090


### 2. Separación de variables predictoras (X) y objetivo (y)

In [46]:
# Separación de variables predictoras (X) y objetivo (y)
X = df.drop(columns=["is_hit","popularity"])
y = df["is_hit"]

display("X shape:", X.shape)
display("X columns:", X.columns.T)
print("Distribución de y:")
display(y.value_counts(normalize=True))


'X shape:'

(232724, 18)

'X columns:'

Index(['genre', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence',
       'duration_min', 'beat_density', 'energy_valence', 'dance_energy',
       'speech_valence', 'acoustic_energy', 'inst_energy', 'dance_valence'],
      dtype='object')

Distribución de y:


is_hit
0    0.954693
1    0.045307
Name: proportion, dtype: float64

### 3. Código: preprocesamiento (OneHot + ColumnTransformer)

In [47]:
# Preservacion de DataFrames
set_config(transform_output="pandas")

# Identificación de columnas numéricas y categóricas
numeric_cols = X.select_dtypes(include=["float64", "int64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

display("Columnas numéricas:", numeric_cols)
print("Columnas categóricas:", categorical_cols)

# Definición del preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)


'Columnas numéricas:'

['acousticness',
 'danceability',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence',
 'duration_min',
 'beat_density',
 'energy_valence',
 'dance_energy',
 'speech_valence',
 'acoustic_energy',
 'inst_energy',
 'dance_valence']

Columnas categóricas: ['genre']


### 4. Código: modelo LightGBM + Pipeline

In [48]:
print(f"✅ Canciones clasificadas como HIT: {df['is_hit'].sum()} de {len(df)} ({df['is_hit'].mean()*100:.2f}%)")
# Correlación directa con popularidad o is_hit
corr = df.corr(numeric_only=True)
corr["is_hit"].sort_values(ascending=False)

✅ Canciones clasificadas como HIT: 10544 de 232724 (4.53%)


is_hit              1.000000
popularity          0.391859
danceability        0.121247
dance_energy        0.108903
loudness            0.108466
dance_valence       0.063807
energy              0.061671
energy_valence      0.049057
valence             0.037736
tempo               0.022765
speech_valence     -0.000522
speechiness        -0.008862
beat_density       -0.012936
duration_min       -0.023957
acoustic_energy    -0.040733
liveness           -0.043567
inst_energy        -0.073886
acousticness       -0.094851
instrumentalness   -0.097980
Name: is_hit, dtype: float64

### 5. Código: train_test_split

70% → train (para entrenar modelos)

10% → validation (para buscar el mejor threshold)

20% → test (solo para evaluar al final)

In [49]:
# 1. Split train/test (80 / 20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 2. Split interno train/val (70 / 10)
X_trainModel, X_val, y_trainModel, y_val = train_test_split(
    X_train, y_train,
    test_size=0.125,        # 0.125 de 80% = 10%
    random_state=42,
    stratify=y_train
)

print(X_trainModel.shape, "→ train (70%)")
print(X_val.shape,        "→ validation (10%)")
print(X_test.shape,       "→ test (20%)")

# 3. Restaurar nombres de columnas 
X_trainModel = pd.DataFrame(X_trainModel, columns=X.columns)
X_val        = pd.DataFrame(X_val,        columns=X.columns)
X_test       = pd.DataFrame(X_test,       columns=X.columns)


(162906, 18) → train (70%)
(23273, 18) → validation (10%)
(46545, 18) → test (20%)


### 6. Pipeline + Grid Search

In [50]:
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1))
])
param_grid = {
    "model__num_leaves": [31,63],
    "model__learning_rate": [0.03,0.015],
    "model__n_estimators": [1000],
    "model__min_child_samples": [20,50],
    "model__scale_pos_weight": [8,12],
}
grid = GridSearchCV(pipeline, param_grid, scoring="f1", cv=3, verbose=1, n_jobs=-1)
grid.fit(X_trainModel, y_trainModel)
grid.best_params_, grid.best_score_

Fitting 3 folds for each of 16 candidates, totalling 48 fits


({'model__learning_rate': 0.03,
  'model__min_child_samples': 50,
  'model__n_estimators': 1000,
  'model__num_leaves': 63,
  'model__scale_pos_weight': 8},
 np.float64(0.544570345938508))

### 7. Evaluación del Modelo (Threshold = 0.5 por defecto)

In [51]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy:   {acc:.4f}")
print(f"F1-score:   {f1:.4f}")
print(f"ROC-AUC:    {auc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy:   0.9535
F1-score:   0.5896
ROC-AUC:    0.9551

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98     44436
           1       0.49      0.74      0.59      2109

    accuracy                           0.95     46545
   macro avg       0.74      0.85      0.78     46545
weighted avg       0.96      0.95      0.96     46545



### 8. Optimización del umbral de decisión (threshold tuning - F1)


In [52]:
# 1. Probabilidades en VALIDATION usando el MEJOR MODELO
y_proba_val = best_model.predict_proba(X_val)[:, 1]

# 2. Threshold tuning usando el set de VALIDACIÓN
thresholds = np.linspace(0.05, 0.95, 30)
f1_scores = []

for t in thresholds:
    preds = (y_proba_val >= t).astype(int)
    f1_scores.append(f1_score(y_val, preds))   

best_t = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

print(f"Mejor threshold (val): {best_t:.3f}")
print(f"Mejor F1-score (val): {best_f1:.4f}")


Mejor threshold (val): 0.640
Mejor F1-score (val): 0.6449


### 9. Evaluación final con threshold optimizado

In [53]:
# 1. Probabilidades en el conjunto de TEST usando el MEJOR MODELO
y_proba_test = best_model.predict_proba(X_test)[:, 1]

# 2. Aplicar threshold óptimo encontrado en VALIDATION
y_pred_opt = (y_proba_test >= best_t).astype(int)

# 3. Calcular métricas sobre TEST
acc_opt = accuracy_score(y_test, y_pred_opt)
f1_opt  = f1_score(y_test, y_pred_opt)
auc_opt = roc_auc_score(y_test, y_proba_test)

print("=== Métricas con threshold optimizado (TEST) ===")
print(f"Usando threshold: {best_t:.3f}")
print(f"Accuracy: {acc_opt:.4f}")
print(f"F1-score: {f1_opt:.4f}")
print(f"ROC-AUC:  {auc_opt:.4f}")


=== Métricas con threshold optimizado (TEST) ===
Usando threshold: 0.640
Accuracy: 0.9691
F1-score: 0.6313
ROC-AUC:  0.9551


### 10. Guardar Modelos Test

In [54]:
# Guardar X_test y y_test para evaluación independiente
X_test.to_csv("../data/processed/X_test.csv", index=False)
pd.DataFrame({"is_hit": y_test}).to_csv("../data/processed/y_test.csv", index=False)

print("Archivos X_test.csv y y_test.csv guardados correctamente.")


Archivos X_test.csv y y_test.csv guardados correctamente.


### 11. Guardado del modelo entrenado

In [55]:

# Definir rutas
PIPELINE_PATH_JOBLIB = "../src/api/models/model_pipeline.joblib"
PIPELINE_PATH_PKL    = "../src/api/models/model_pipeline.pkl"

# Guardar pipeline
joblib.dump(best_model, PIPELINE_PATH_JOBLIB)

with open(PIPELINE_PATH_PKL, "wb") as f:
    pickle.dump(best_model, f)

print("Modelos guardados correctamente.")


Modelos guardados correctamente.
